<a href="https://colab.research.google.com/github/juanmed/dw_a/blob/main/DLL_package_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 3.0MB 53.1MB/s 
     |████████████████████████████████| 460kB 42.1MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624302 sha256=aaa756c180a948fb2696de26040e565486bcbef38c2e6926c7a6c2b734186217
  Stored in directory: /tmp/pip-ephem-wheel-cache-x4f70q4w/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=77d17034185fc3a8501dcc44dfe293a3337d297f5c32b19aa5b1e59017a8d51f
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=0b81106bc4e15afde75959aea0998e07752477c8f5dff451f5a4add98caab176
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=8552ffb2c4b33350e68d52fab5f0fbce1323a16c3674519222e6

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


In [4]:
import pathlib as pl

training_path = "./workspace/training_demo"
annotations_path = "./annotations"

if not os.path.exists(training_path+"/images/train"):
  pl.Path(training_path+"/images/train").mkdir(parents=True, exist_ok=True)
if not os.path.exists(training_path+"/test"):
  pl.Path(training_path+"/images/test").mkdir(parents=True, exist_ok=True)
if not os.path.exists(annotations_path):
  pl.Path(annotations_path).mkdir(parents=True, exist_ok=True)

In [5]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=980e27e22abb8c5470f1bb486dfe96f2ed1a57197fa745fa255489026b084cae
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [6]:
import wget
dataset_partition_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/dataset_partition.py"
dataset_partition_file = wget.download(dataset_partition_url)

In [7]:
!pip install gdown

In [8]:
#!cd /content/workspace/training_demo/images/data/ | rm *


In [9]:
# get images
#images_url = "https://drive.google.com/file/d/1s04AS2prwhbgZmzjvype_HySfxZsUYgP/view?usp=sharing"
import gdown
import shutil
url = 'https://drive.google.com/uc?id=1s04AS2prwhbgZmzjvype_HySfxZsUYgP'
output = 'image_pack.zip'
gdown.download(url, output, quiet=False)
shutil.move(output, training_path+"/images/"+output)


Downloading...
From: https://drive.google.com/uc?id=1s04AS2prwhbgZmzjvype_HySfxZsUYgP
To: /content/image_pack.zip
998MB [00:11, 83.3MB/s]


'./workspace/training_demo/images/image_pack.zip'

In [10]:
!pip install zipfile36

In [11]:
import sys
if sys.version_info >= (3, 6):
    import zipfile
else:
    import zipfile36 as zipfile
# Create a ZipFile Object and load sample.zip in it
with zipfile.ZipFile(training_path+"/images/"+output, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(training_path+"/images/")

In [12]:
#!rm ./workspace/training_demo/images/data/pkrecog_016_0029.jpg

In [13]:
# Partition the dataset
%%bash
python dataset_partition.py -x -i "./workspace/training_demo/images/data" -o "./workspace/training_demo/images/" -r 0.1

In [14]:
gen_tf_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/generate_tfrecord.py"
wget.download(gen_tf_url)

'generate_tfrecord.py'

In [15]:

labelmap_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/label_map.pbtxt"
labelmap_file = wget.download(labelmap_url)

anno_path = "./workspace/training_demo/annotations"
if not os.path.exists(anno_path):
  pl.Path(anno_path).mkdir(parents=True, exist_ok=True)

shutil.move(labelmap_file, anno_path+"/"+labelmap_file)

'./workspace/training_demo/annotations/label_map.pbtxt'

In [16]:
# Generate .record files
%%bash
python generate_tfrecord.py -x ./workspace/training_demo/images/train -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/train.record
python generate_tfrecord.py -x ./workspace/training_demo/images/test -l ./workspace/training_demo/annotations/label_map.pbtxt -o ./workspace/training_demo/annotations/test.record

Successfully created the TFRecord file: ./workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: ./workspace/training_demo/annotations/test.record


In [17]:
# Download pretrained EfficienteNet D5 1280x1280 from
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


In [18]:

import sys
net_name = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"
#effnet_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz"
effnet_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz"
ssdnet_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"
filename = wget.download(ssdnet_url)

In [19]:
import tarfile

pretrained_path = "./workspace/training_demo/pre-trained-models"
if not os.path.exists(pretrained_path):
  pl.Path(pretrained_path).mkdir(parents=True, exist_ok=True)

tar = tarfile.open(filename) 
tar.extractall(path=pretrained_path) # untar file into same directory
tar.close()

In [20]:
# Configure model
models_path = "./workspace/training_demo/models/" + net_name
if not os.path.exists(models_path):
  pl.Path(models_path).mkdir(parents=True, exist_ok=True)

config_url = "https://raw.githubusercontent.com/juanmed/dw_a/main/dll_package_recognition/{}.config".format(net_name)
config_file = wget.download(config_url)

shutil.move(config_file, models_path + "/" + config_file)
shutil.copy("./models/research/object_detection/model_main_tf2.py","./workspace/training_demo/model_main_tf2.py")

'./workspace/training_demo/model_main_tf2.py'

In [21]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [22]:
#Setup TensorBoard
#from this example: https://colab.research.google.com/drive/1afN2SALDooZIHbBGmWZMT6cZ8ccVElWk#scrollTo=DZO3EUaPea1E

In [23]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-03-07 20:12:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.171.215.128, 34.202.112.21, 3.216.182.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.171.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  36.9MB/s    in 0.4s    

2021-03-07 20:12:06 (36.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [24]:
LOG_DIR = '/content/workspace/training_demo/models/'+net_name
print(LOG_DIR)
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

/content/workspace/training_demo/models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8


In [25]:
get_ipython().system_raw('./ngrok http 6006 &')

In [27]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://3e763e50b3e6.ngrok.io


In [ ]:
# Start training
%%bash
cd workspace/training_demo/
python model_main_tf2.py --model_dir=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 --pipeline_config_path=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config #--num_train_steps=100 --sample_1_of_n_eval_examples=5

In [ ]:
!pwd


In [ ]:
# Export Inference graph for later inference
shutil.copy("./models/research/object_detection/exporter_main_v2.py","./workspace/training_demo/exporter_main_v2.py")

In [ ]:
%%bash
cd ./workspace/training_demo
python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config --trained_checkpoint_dir /content/workspace/training_demo/models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ --output_directory ./exported-models/my_model

In [ ]:
dir_name = "/content/workspace/training_demo/exported-models"
output_filename="/content/package_detection_ssd"
shutil.make_archive(output_filename, 'zip', dir_name)

In [29]:
# Install COCO evaluation tools
%%bash 
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
make
cp -r pycocotools ../../models/research/

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/

Cloning into 'cocoapi'...
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
                                                      ^
../common/maskApi.c:167:3: warning: this ‘for’ clause does 

In [30]:
# Start evaluation
%%bash
cd workspace/training_demo/
python model_main_tf2.py --model_dir=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 --pipeline_config_path=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config --checkpoint_dir=models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

Process is terminated.


In [31]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


#Test

In [ ]:
aasdfa
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [ ]:

tf.keras.backend.clear_session()
print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 1
pipeline_config = models_path + "/" + config_file
checkpoint_path = './workspace/training_demo/pre-trained-models/efficientdet_d5_coco17_tpu-32/checkpoint/ckpt-0'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
num_classes = 5
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

In [ ]:
# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

In [ ]:
tf.keras.backend.set_learning_phase(True)
# These parameters can be tuned; since our training set has 5 images
# it doesn't make sense to have a much larger batch size, though we could
# fit more examples in memory if we wanted to.
batch_size = 4
learning_rate = 0.01
num_batches = 100

# Select variables in top layers to fine-tune.
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
  if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
    to_fine_tune.append(var)

In [ ]:
# Set up forward + backward pass for a single train step.
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
  """Get a tf.function for training step."""

  # Use tf.function for a bit of speed.
  # Comment out the tf.function decorator if you want the inside of the
  # function to run eagerly.
  @tf.function
  def train_step_fn(image_tensors,
                    groundtruth_boxes_list,
                    groundtruth_classes_list):
    """A single training iteration.

    Args:
      image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    with tf.GradientTape() as tape:
      preprocessed_images = tf.concat(
          [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_tensors], axis=0)
      prediction_dict = model.predict(preprocessed_images, shapes)
      losses_dict = model.loss(prediction_dict, shapes)
      total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
      gradients = tape.gradient(total_loss, vars_to_fine_tune)
      optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    return total_loss

  return train_step_fn

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(
    detection_model, optimizer, to_fine_tune)


In [ ]:
print('Start fine-tuning!', flush=True)
for idx in range(num_batches):
  # Grab keys for a random subset of examples
  all_keys = list(range(len(train_images_np)))
  random.shuffle(all_keys)
  example_keys = all_keys[:batch_size]

  # Note that we do not do data augmentation in this demo.  If you want a
  # a fun exercise, we recommend experimenting with random horizontal flipping
  # and random cropping :)
  gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
  gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
  image_tensors = [train_image_tensors[key] for key in example_keys]

  # Training step (forward pass + backwards pass)
  total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

  if idx % 10 == 0:
    print('batch ' + str(idx) + ' of ' + str(num_batches)
    + ', loss=' +  str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')